# 02-setup-milvus

起動済みの Milvus に対して、スクレイピングした ThinkIT の記事データを格納するためのノートブックです。

In [ ]:
import os
import glob
import numpy as np
from dotenv import load_dotenv

from pymilvus import MilvusClient, DataType

from oci.auth.signers import InstancePrincipalsSecurityTokenSigner
from oci.generative_ai_inference.generative_ai_inference_client import GenerativeAiInferenceClient
from oci.generative_ai_inference.models import EmbedTextDetails, OnDemandServingMode

from langchain_cohere.embeddings import CohereEmbeddings
from langchain_community.embeddings.oci_generative_ai import OCIGenAIEmbeddings

from langchain_text_splitters import CharacterTextSplitter

from langfuse import Langfuse
from langfuse.decorators import observe

In [ ]:
load_dotenv()

uri = os.getenv("MILVUS_URI")
compartment_id = os.getenv("COMPARTMENT_ID")
cohere_api_key = os.getenv("COHERE_API_KEY")
endpoint = os.getenv("GENAI_ENDPOINT")
embedding_batch_size = 96
collection_name = "ochacafe_collection"
langfuse_secret_key = os.getenv("LANGFUSE_SECRET_KEY")
langfuse_public_key = os.getenv("LANGFUSE_PUBLIC_KEY")
langfuse_host = os.getenv("LANGFUSE_HOST")

client = MilvusClient(uri=uri)
oci_generative_ai_inference_client = GenerativeAiInferenceClient(config={}, signer=InstancePrincipalsSecurityTokenSigner(), service_endpoint=endpoint)
oci_embedding = OCIGenAIEmbeddings(
    compartment_id=compartment_id,
    model_id="cohere.embed-multilingual-v3.0",
    service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com",
    auth_type="API_KEY",
    auth_profile="CHICAGO"
)
cohere_embedding = CohereEmbeddings(
    cohere_api_key=cohere_api_key,
    model="embed-multilingual-v3.0"
)
langfuse = Langfuse(
    secret_key=langfuse_secret_key,
    public_key=langfuse_public_key,
    host=langfuse_host
)

In [ ]:
client.drop_collection(collection_name=collection_name)

In [ ]:
schema = MilvusClient.create_schema(
    auto_id=True,
    enable_dynamic_field=True,
)
schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True)
schema.add_field(field_name="vector", datatype=DataType.FLOAT_VECTOR, dim=1024)
schema.add_field(field_name="text", datatype=DataType.VARCHAR, max_length=2048)

client.create_collection(
    collection_name=collection_name,
    schema=schema
)

In [ ]:
@observe()
def embedding_oci(inputs: list[str]) -> list[list[float]]:
    res = oci_generative_ai_inference_client.embed_text(
        embed_text_details=EmbedTextDetails(
            compartment_id=compartment_id,
            inputs=inputs,
            input_type="SEARCH_DOCUMENT",
            serving_mode=OnDemandServingMode(
                model_id="cohere.embed-multilingual-v3.0"
            )
        )
    )
    if res.status != 200:
        exit(1)
    return res.data.embeddings

# embedding_oci(["こんにちは", "ありがとう"])

@observe()
def embedding_cohere(inputs: list[str]) -> list[list[float]]:
    res = cohere_embedding.embed_documents(
        texts=inputs
    )
    return res

# embedding_cohere(["こんにちは", "ありがとう"])

@observe()
def embedding(inputs: list[str]) -> list[list[float]]:
    res = oci_embedding.embed_documents(texts=inputs)
    return res

# embedding(["こんにちは", "ありがとう"])

In [ ]:
@observe()
def create_data_by_character_text_splitter(content: str) -> list[dict]:
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=200,
        chunk_overlap=20, # 10%
        length_function=len,
        is_separator_regex=False,
    )
    texts = text_splitter.create_documents([content])
    inputs = [text.page_content for text in texts]
    print("inputs length: ", len(inputs))
    if len(inputs) > embedding_batch_size:
        vectors = []
        inputs_ary = np.array_split(inputs, len(inputs)//embedding_batch_size + 1)
        print("split array size: ", len(inputs_ary))
        for input in inputs_ary:
            # vectors.append([embed for embed in embedding_cohere(inputs=input.tolist())])
            print("input size: ", len(input))
            vectors.extend([embed for embed in embedding_cohere(inputs=input.tolist())])
            print("vectors length: ", len(vectors))
    else:
        vectors = embedding_cohere(inputs=inputs)
    data = [
        {"vector": vectors[i], "text": inputs[i], "subject": "technology"} # enabled auto-id
        for i in range(len(vectors))
    ]
    return data

# create_data_by_character_text_splitter("""
# 「Oracle Cloud Hangout Cafe」コミュニティとは
# 「Oracle Cloud Hangout Cafe」(通称「おちゃかふぇ」/以降、OCHaCafe)は、日本オラクルが主催するコミュニティの1つです。定期的に、開発者・エンジニアに向けたクラウドネイティブな時代に身につけておくべきテクノロジーを深堀する勉強会を開催しています。
# OCHaCafeのテーマは、Oracleの製品や技術に特化せず、オープン/デファクト・スタンダードを基準に選定して、参加される方々がいかなるプラットフォームを利用している場合でも、必ずスキルアップに役立てられる内容を趣旨としています。
# 本連載について
# OCHaCafeは、2018年12月から開始して、この4年間、多くの方にご参加いただきました。そして、この4年の間にたくさんの有益なコンテンツが蓄積されました。本連載では、その中からご好評いただいたテーマを選択して、ダイジェスト(全6回)としてお送りします。

# 第1回「CI/CD最新事情」第2回「マイクロサービスの認証・認可とJWT」第3回「Kubernetes Operator 超入門」第4回「Kubernetes のセキュリティ」第5回「実験！カオスエンジニアリング」第6回「Service Mesh がっつり入門！」

# なお、本連載にあたり、各回の内容は実施時の発表内容をベースとしますが、技術の進化により執筆時に内容を変更する場合もあります。その際は、実施時の発表内容とは異なりますので予めご了承ください。
# 連載第1回目の今回は、2021年6月9日に開催された「Oracle Cloud Hangout Cafe Season4 #3『CI/CD 最新事情』」の発表内容に基づいて紹介していきます。
# この発表では、2021年6月時点のCI/CD最新事情として、CI/CDのこれまで、今、これからという時間軸を設けて、CI/CDのおさらいからCIOps、GitOps、Progressive Deliveryまで、デモを交えながら辿りました。
# アジェンダ
# 今回は、以下アジェンダの「CI/CD これまでを見る」「CI/CD 今を見る」「CI/CD これからを見る」を中心に紹介します。

# ・ CI/CD これまでを見る
#  　- CIOps
# 　　- CI/CD おさらい
# 　　- CIOps について
# ・ CI/CD 今を見る
# 　- GitOps
# 　　- GitOps について
# 　　- CI & GitOps
# 　　- 手元で始める GitOps (デモ)
# ・ CI/CD これからを見る
# 　- Progressive Delivery
# 　　- Progressive Delivery について
# ・ Oracle's DevOps
# 　- OCI DevOps Service
# 　- OKE & DevOps
# ・ 最後に
# ・ 参考資料

# 発表資料と動画については、下記のリンクを参照してください。
# 【資料リンク】【動画リンク】
# CI/CD これまでを見る
# 最初に、CI(Continuous Integration/継続的インテグレーション)とCD(Continuous Delivery/継続的デリバリー)について整理します。その上で、CIOpsを理解します。
# CIOps
# CI/CDおさらい
# 1. CIとは?CIは、アプリケーション開発に必要となるビルド、テスト、成果物(アーティファクト)の保存までの工程を自動化して、継続的に行われる状態にすることです。
# コンテナアプリケーション開発における例としては、ソースコードの変更を契機にアプリケーションビルド、アプリケーションテスト、コンテナイメージビルド、コンテナイメージレジストリへのコンテナイメージ格納までの工程を示す場合が多いとされています。
# CI環境を整備することで、早期バグの発見による品質向上や検証時間の短縮によるリリーススピードの向上が見込まれ、開発者が開発に集中できる環境でより良い開発サイクルを実現します。
# 2. CDとは?CDは、CIによって生み出された成果物(アーティファクト)を、実際の環境(ステージングやプロダクション)へ自動的に配置することです。
# コンテナアプリケーション開発における例としては、CIによってコンテナイメージレジストリに格納されたコンテナイメージをデプロイ先となるコンテナプラットフォームに適用する工程です。
# CD環境を整備することで、エンドユーザに影響を与えることなく、安全にアプリケーションのデプロイを実現します。
# 3. CI/CDとは?これまでの流れを整理すると、CI/CDはソースコードの更新を契機にビルド、テストが進行して、アプリケーションが対象となる環境にデプロイされるまでの工程を自動化することだと理解できます。


# CI/CD全体図

# 4. CI/CDのメリットアプリケーション開発において、CI/CDを導入するメリットはいくつかあります。以下に挙げた主な例を基に見ていきましょう。

# リリースサイクルの短縮により、市場ニーズに早く対応できるエンドユーザからのフィードバックを受けて、実装、ビルド、テスト、リリースまでの工程を迅速に実現できます。市場のニーズに早く対応できることで、エンドユーザの満足度、企業競争力や収益の向上も見込めます。
# テスト工程が組まれていることにより、アプリケーションの品質を担保できる早期にバグを発見できることにより品質の向上を計れます。
# 各工程が自動化されたパイプラインのため、全体的な工数削減を見込める各工程が自動化されることによって手動作業が減って工数の削減を見込めます。そして、手動作業が減ることによってヒューマンエラーの回避にもつながります。
# 開発者は開発ロジックに集中できるため、開発効率の向上を見込める開発者が、自動化の恩恵によって、アプリケーション開発ロジックに専念できるようになり、より良い開発サイクルを実現できます。

# CI/CD環境を整備することで、早く正確により良いアプリケーションサービスをエンドユーザに届けることが可能となり、満足度の向上から企業競争力や収益の向上につながります。
# 5. CI/CDの難しさ実際にCI/CDの導入を検討すると、状況にもよりますが簡単にはいかないところもあります。以下に挙げた主な例を基に見ていきましょう。

# CI/CD環境の構築、運用に乗せるまでの試行錯誤実際にCI/CD環境を構築して運用していくことを考えると課題は避けられないと思われます。例えば、開発するアプリケーションの特性、基盤/開発/運用の体制や人数、教育など検討事項は多くあります。
# 多種多様で豊富なCI/CDツールの技術選定CI/CDを実現するツールやサービスはたくさんあるため、基盤/開発/運用者の体制やスキルセットに合わせて検討が必要となります。
# コストの検討開発規模、体制人数、サポートなど状況に合わせて検討が必要となります。
# 定期的なCI/CD環境のメンテナスCI/CD環境は、一度構築して終わりではなく、維持、持続、改善をどうしていくか検討が必要となります。

# もちろん、ここに挙げたことだけではありません。CI/CDの導入において時間やコストも要することは避けられないと思われますが、長い目で見たときに、その分のメリットをどう見込むかが大切なことではないでしょうか。
# CIOpsについて
# CI/CDのおさらいをしたところで、次はCIOpsをベースにコンテナアプリケーション開発におけるCI/CDを見ていきます。
# CIOpsは、これまでのアプリケーション開発からコンテナアプリケーション開発において、広く利用されている手法です。OSS、各ベンダーやクラウドプロバイダーが提供するツールやマネージドサービスがあります。こうしたツールを組み合わせたり、サービスを利用してCI/CD環境を構築します。
# ・プッシュ(Push)型CIOpsは、CI/CDツールやサービスがソースコードの更新を契機に、ビルド、テスト、デプロイの各工程を自動実行します。
# 下図は、コンテナアプリケーション開発において、ソースコード更新してプッシュやマージ処理などをトリガーにCIパイプラインが稼働して、ビルド、テスト、コンテナイメージビルド、コンテナイメージプッシュ、CDパイプラインによってデプロイされる流れです。


# CIOps全体図

# CIOpsは、CI/CDツールやサービスから対象となるプラットフォームにデプロイすることから、プッシュ(Push)型と呼ばれます。デプロイする上でCI/CDツール側に必要となる権限の付与やデプロイ先が増えるごとにその権限を管理するため、セキュリティを十分に考慮する必要があります。
# CI/CD 今を見る
# CIOpsは、CI/CDツールやサービスを利用して、対象となるプラットフォームにデプロイするプッシュ(Push)型でした。ここからは、Kubernetesを基盤としたコンテナプラットフォームが前提となるGitOpsを見ていきます。
# GitOps
# GitOpsについて
# GitOpsという手法が流布し始めたのは、2017年8月に公開されたWeaveworksの「Operations by Pull Request」ブログ記事だと言われています。GitOpsのガイドラインを整理すると、以下となります。

# コードとして記述できるものは、すべてGitに保存及び管理するKubernetesクラスタ上で稼働するコンテナアプリケーションを前提Kubernetes CLIであるkubectlコマンドを直接使用しないKubernetesのオペレータパターンに従うコントローラーを使用

# GitOpsは、デプロイ対象となるプラットフォームがKubernetes基盤上で稼働するコンテナアプリケーションであることが前提で、手動によるコマンド操作をせずにGitを利用してCI/CDを実現する手法です。次項で具体的に見ていきます。
# ・プル(Pull)型GitOpsは、アプリケーションのソースコードを管理するCodeリポジトリとKubernetesで使用するマニフェストを管理するConfigリポジトリを分けて管理します。そして、Kubernetesクラスタ内に配備されたGitOpsオペレータがConfigリポジトリを定期的にポーリングして、マニフェストの更新を検知すると自動的にその更新内容をKubernetesクラスタに適用する仕組みです。
# 下図は、GitOpsの流れです。

# 開発者がソースコードを更新して、Codeリポジトリにプッシュを実行CIパイプラインが稼働して、ビルド、テスト、コンテナイメージビルド、コンテナイメージプッシュの自動実行コンテナイメージに付与された新たなタグをマニフェストに反映後、Configリポジトリにプルリクエスト開発者がプルリクエストをマージGitOpsオペレータが更新を検知後、Kubernetesクラスタに適用



# GitOps全体図

# GitOpsは、Kubernetesを基盤とするコンテナプラットフォームからGitOpsオペレータによってデプロイを実行するため、CIOpsのプッシュ(Push)型とは異なり、プル(Pull)型であることが特徴です。
# GitOpsオペレータは、後述するArgo CDやFluxなどのGitOpsツールもあります。
# ・GitOpsの特徴CodeとConfigリポジトリを分けることで、相互の影響を回避して境界線を明確にできます。また、デプロイに必要となるクレデンシャルなどを外部に持たせることも無いのでセキュリティリスクや管理の煩雑性も抑えられます。


# GitOps特徴(1)

# KubernetesのマニフェストをGitで管理するということは、Configリポジトリにあるマニフェストの内容と実環境の状態が同一になります。Git管理により、マニフェストの履歴管理やロールバック対応など柔軟に実施できるため、実環境の再現性も保証できます。


# GitOps特徴(2)

# マニフェストをGitで管理することによって得られる主な特徴を挙げます。

# デプロイ先に対して「誰が、何時、何」を変更したのか(変更差分)を履歴で追えるデプロイ先をいつでも前の状態に戻すこと(ロールバック)ができるプルリクエストによるレビュー・マージプロセスを通すことで組織ガバナンスを適用できる

# 自動化という点では、CIOpsと同様に手動コマンド操作によるヒューマンエラーの回避、運用コストの低減を見込めます。


  

# 「Oracle Cloud Hangout Cafe」コミュニティとは
# 「Oracle Cloud Hangout Cafe」(通称「おちゃかふぇ」/以降、OCHaCafe)は、日本オラクルが主催するコミュニティの1つです。定期的に、開発者・エンジニアに向けたクラウドネイティブな時代に身につけておくべきテクノロジーを深堀する勉強会を開催しています。
# OCHaCafeのテーマは、Oracleの製品や技術に特化せず、オープン/デファクト・スタンダードを基準に選定して、参加される方々がいかなるプラットフォームを利用している場合でも、必ずスキルアップに役立てられる内容を趣旨としています。
# 本連載について
# OCHaCafeは、2018年12月から開始して、この4年間、多くの方にご参加いただきました。そして、この4年の間にたくさんの有益なコンテンツが蓄積されました。本連載では、その中からご好評いただいたテーマを選択して、ダイジェスト(全6回)としてお送りします。

# 第1回「CI/CD最新事情」第2回「マイクロサービスの認証・認可とJWT」第3回「Kubernetes Operator 超入門」第4回「Kubernetes のセキュリティ」第5回「実験！カオスエンジニアリング」第6回「Service Mesh がっつり入門！」

# なお、本連載にあたり、各回の内容は実施時の発表内容をベースとしますが、技術の進化により執筆時に内容を変更する場合もあります。その際は、実施時の発表内容とは異なりますので予めご了承ください。
# 連載第1回目の今回は、2021年6月9日に開催された「Oracle Cloud Hangout Cafe Season4 #3『CI/CD 最新事情』」の発表内容に基づいて紹介していきます。
# この発表では、2021年6月時点のCI/CD最新事情として、CI/CDのこれまで、今、これからという時間軸を設けて、CI/CDのおさらいからCIOps、GitOps、Progressive Deliveryまで、デモを交えながら辿りました。
# アジェンダ
# 今回は、以下アジェンダの「CI/CD これまでを見る」「CI/CD 今を見る」「CI/CD これからを見る」を中心に紹介します。

# ・ CI/CD これまでを見る
#  　- CIOps
# 　　- CI/CD おさらい
# 　　- CIOps について
# ・ CI/CD 今を見る
# 　- GitOps
# 　　- GitOps について
# 　　- CI & GitOps
# 　　- 手元で始める GitOps (デモ)
# ・ CI/CD これからを見る
# 　- Progressive Delivery
# 　　- Progressive Delivery について
# ・ Oracle's DevOps
# 　- OCI DevOps Service
# 　- OKE & DevOps
# ・ 最後に
# ・ 参考資料

# 発表資料と動画については、下記のリンクを参照してください。
# 【資料リンク】【動画リンク】
# CI/CD これまでを見る
# 最初に、CI(Continuous Integration/継続的インテグレーション)とCD(Continuous Delivery/継続的デリバリー)について整理します。その上で、CIOpsを理解します。
# CIOps
# CI/CDおさらい
# 1. CIとは?CIは、アプリケーション開発に必要となるビルド、テスト、成果物(アーティファクト)の保存までの工程を自動化して、継続的に行われる状態にすることです。
# コンテナアプリケーション開発における例としては、ソースコードの変更を契機にアプリケーションビルド、アプリケーションテスト、コンテナイメージビルド、コンテナイメージレジストリへのコンテナイメージ格納までの工程を示す場合が多いとされています。
# CI環境を整備することで、早期バグの発見による品質向上や検証時間の短縮によるリリーススピードの向上が見込まれ、開発者が開発に集中できる環境でより良い開発サイクルを実現します。
# 2. CDとは?CDは、CIによって生み出された成果物(アーティファクト)を、実際の環境(ステージングやプロダクション)へ自動的に配置することです。
# コンテナアプリケーション開発における例としては、CIによってコンテナイメージレジストリに格納されたコンテナイメージをデプロイ先となるコンテナプラットフォームに適用する工程です。
# CD環境を整備することで、エンドユーザに影響を与えることなく、安全にアプリケーションのデプロイを実現します。
# 3. CI/CDとは?これまでの流れを整理すると、CI/CDはソースコードの更新を契機にビルド、テストが進行して、アプリケーションが対象となる環境にデプロイされるまでの工程を自動化することだと理解できます。


# CI/CD全体図

# 4. CI/CDのメリットアプリケーション開発において、CI/CDを導入するメリットはいくつかあります。以下に挙げた主な例を基に見ていきましょう。

# リリースサイクルの短縮により、市場ニーズに早く対応できるエンドユーザからのフィードバックを受けて、実装、ビルド、テスト、リリースまでの工程を迅速に実現できます。市場のニーズに早く対応できることで、エンドユーザの満足度、企業競争力や収益の向上も見込めます。
# テスト工程が組まれていることにより、アプリケーションの品質を担保できる早期にバグを発見できることにより品質の向上を計れます。
# 各工程が自動化されたパイプラインのため、全体的な工数削減を見込める各工程が自動化されることによって手動作業が減って工数の削減を見込めます。そして、手動作業が減ることによってヒューマンエラーの回避にもつながります。
# 開発者は開発ロジックに集中できるため、開発効率の向上を見込める開発者が、自動化の恩恵によって、アプリケーション開発ロジックに専念できるようになり、より良い開発サイクルを実現できます。

# CI/CD環境を整備することで、早く正確により良いアプリケーションサービスをエンドユーザに届けることが可能となり、満足度の向上から企業競争力や収益の向上につながります。
# 5. CI/CDの難しさ実際にCI/CDの導入を検討すると、状況にもよりますが簡単にはいかないところもあります。以下に挙げた主な例を基に見ていきましょう。

# CI/CD環境の構築、運用に乗せるまでの試行錯誤実際にCI/CD環境を構築して運用していくことを考えると課題は避けられないと思われます。例えば、開発するアプリケーションの特性、基盤/開発/運用の体制や人数、教育など検討事項は多くあります。
# 多種多様で豊富なCI/CDツールの技術選定CI/CDを実現するツールやサービスはたくさんあるため、基盤/開発/運用者の体制やスキルセットに合わせて検討が必要となります。
# コストの検討開発規模、体制人数、サポートなど状況に合わせて検討が必要となります。
# 定期的なCI/CD環境のメンテナスCI/CD環境は、一度構築して終わりではなく、維持、持続、改善をどうしていくか検討が必要となります。

# もちろん、ここに挙げたことだけではありません。CI/CDの導入において時間やコストも要することは避けられないと思われますが、長い目で見たときに、その分のメリットをどう見込むかが大切なことではないでしょうか。
# CIOpsについて
# CI/CDのおさらいをしたところで、次はCIOpsをベースにコンテナアプリケーション開発におけるCI/CDを見ていきます。
# CIOpsは、これまでのアプリケーション開発からコンテナアプリケーション開発において、広く利用されている手法です。OSS、各ベンダーやクラウドプロバイダーが提供するツールやマネージドサービスがあります。こうしたツールを組み合わせたり、サービスを利用してCI/CD環境を構築します。
# ・プッシュ(Push)型CIOpsは、CI/CDツールやサービスがソースコードの更新を契機に、ビルド、テスト、デプロイの各工程を自動実行します。
# 下図は、コンテナアプリケーション開発において、ソースコード更新してプッシュやマージ処理などをトリガーにCIパイプラインが稼働して、ビルド、テスト、コンテナイメージビルド、コンテナイメージプッシュ、CDパイプラインによってデプロイされる流れです。


# CIOps全体図

# CIOpsは、CI/CDツールやサービスから対象となるプラットフォームにデプロイすることから、プッシュ(Push)型と呼ばれます。デプロイする上でCI/CDツール側に必要となる権限の付与やデプロイ先が増えるごとにその権限を管理するため、セキュリティを十分に考慮する必要があります。
# CI/CD 今を見る
# CIOpsは、CI/CDツールやサービスを利用して、対象となるプラットフォームにデプロイするプッシュ(Push)型でした。ここからは、Kubernetesを基盤としたコンテナプラットフォームが前提となるGitOpsを見ていきます。
# GitOps
# GitOpsについて
# GitOpsという手法が流布し始めたのは、2017年8月に公開されたWeaveworksの「Operations by Pull Request」ブログ記事だと言われています。GitOpsのガイドラインを整理すると、以下となります。

# コードとして記述できるものは、すべてGitに保存及び管理するKubernetesクラスタ上で稼働するコンテナアプリケーションを前提Kubernetes CLIであるkubectlコマンドを直接使用しないKubernetesのオペレータパターンに従うコントローラーを使用

# GitOpsは、デプロイ対象となるプラットフォームがKubernetes基盤上で稼働するコンテナアプリケーションであることが前提で、手動によるコマンド操作をせずにGitを利用してCI/CDを実現する手法です。次項で具体的に見ていきます。
# ・プル(Pull)型GitOpsは、アプリケーションのソースコードを管理するCodeリポジトリとKubernetesで使用するマニフェストを管理するConfigリポジトリを分けて管理します。そして、Kubernetesクラスタ内に配備されたGitOpsオペレータがConfigリポジトリを定期的にポーリングして、マニフェストの更新を検知すると自動的にその更新内容をKubernetesクラスタに適用する仕組みです。
# 下図は、GitOpsの流れです。

# 開発者がソースコードを更新して、Codeリポジトリにプッシュを実行CIパイプラインが稼働して、ビルド、テスト、コンテナイメージビルド、コンテナイメージプッシュの自動実行コンテナイメージに付与された新たなタグをマニフェストに反映後、Configリポジトリにプルリクエスト開発者がプルリクエストをマージGitOpsオペレータが更新を検知後、Kubernetesクラスタに適用



# GitOps全体図

# GitOpsは、Kubernetesを基盤とするコンテナプラットフォームからGitOpsオペレータによってデプロイを実行するため、CIOpsのプッシュ(Push)型とは異なり、プル(Pull)型であることが特徴です。
# GitOpsオペレータは、後述するArgo CDやFluxなどのGitOpsツールもあります。
# ・GitOpsの特徴CodeとConfigリポジトリを分けることで、相互の影響を回避して境界線を明確にできます。また、デプロイに必要となるクレデンシャルなどを外部に持たせることも無いのでセキュリティリスクや管理の煩雑性も抑えられます。


# GitOps特徴(1)

# KubernetesのマニフェストをGitで管理するということは、Configリポジトリにあるマニフェストの内容と実環境の状態が同一になります。Git管理により、マニフェストの履歴管理やロールバック対応など柔軟に実施できるため、実環境の再現性も保証できます。


# GitOps特徴(2)

# マニフェストをGitで管理することによって得られる主な特徴を挙げます。

# デプロイ先に対して「誰が、何時、何」を変更したのか(変更差分)を履歴で追えるデプロイ先をいつでも前の状態に戻すこと(ロールバック)ができるプルリクエストによるレビュー・マージプロセスを通すことで組織ガバナンスを適用できる

# 自動化という点では、CIOpsと同様に手動コマンド操作によるヒューマンエラーの回避、運用コストの低減を見込めます。


 


# CI & GitOps
# 参考までにGitOps環境を構築する上で利用できる主なCIツールを例挙しておきます。CIOpsにおいても利用され続けているCIツールとGitOpsを組み合わせるケースとして主なツールは以下です。

# GitHub ActionsGitLab RunnerCircle CITravis CI

# KubernetesのCustomResourceとしてCIパイプラインをマニフェストで定義できるKubernetesネイティブなツールは以下です。

# TEKTONArgo Workflows

# 主なGitOpsツールは、以下です。

# Argo CDFluxPipeCDJenkis X

# Argoには、Argo Workflows、Argo CD以外にもGitOpsの利便性を高めるツール群もあるので、同じファミリー(Argo Family)として組み合わせて利用できます。
# こうしたツールを組み合わせることでGitOps環境を構築できます。
# ・CI & CD Securityここでは、GitOpsにおいてパイプラインに組み込む、主なセキュリティ対策ナレッジを紹介します。
# 技術の進化に伴い様々なナレッジがありますので、あくまでも一例と捉えてください。
# 1. コンテナイメージビルドコンテナイメージのビルドは、CIパイプラインに組み込む上でセキュアで高速であることが望ましいです。こうした要望に対応するビルドツールもいくつかあります。ここでは、BuildKitとKanikoを紹介します。
# BuildKitは、Dockerfileの命令を並列実行、キャッシュ判定の仕組みで高速にビルドを実行します。そして、非Root / 非Privilegedな実行、DockerfileのRUN 命令を実行する際にseccomp*1を有効化する仕組みによってセキュアにビルドできます。
# Kanikoは、OSやコンテナランタイムに依存せずにコンテナイメージをビルドする場合に有用です。コンテナ内のユーザ空間でビルドすることで、CIで推奨されるDooD*2でのRoot権限で稼働しているホスト上へのセキュリティリスクを回避できます。
# *1:コンテナ内のプロセスが呼び出せるシステムコールを制限できるLinuxカーネルの機能*2:Docker outside of Docker。起動したコンテナからホストのDockerデーモンでイメージビルドする方法
# 2. イメージスキャンコンテナイメージもセキュアであることが望まれます。マネージドサービスのコンテナイメージレジストリに脆弱性検査機能が実装されていますが、CI時に脆弱性スキャンをかけて検査したコンテナイメージをレジストリにプッシュすることでセキュリティを高めることもできます。ここでは、Trivyとdockleを紹介します。
# Trivyは、脆弱性データベースからコンテナイメージを診断します。診断結果をレポートとして出力できます。ワンバイナリでCIに組み込みやすいのも特徴です。
# dockleは、CIS BenchmarkのDockerに関する項目、Dockerfileのベストプラクティスを基に検査できるツールです。そのため、Trivy では検知できない脆弱性を検査できます。
# 3. ポリシーチェックKubernetesのリソースは、マニフェストに定義して登録します。そのマニフェストの内容が正しいか、許可できるものかをチェックしてセキュリティを高める必要があります。
# GitOpsではCI時にConftestというツールを利用してマニフェストをチェックできます。さらに、CD時にGatekeeperというツールを利用してKubernetesクラスタ登録時にチェックできます。その際、ポリシーチェックに利用するポリシーエンジンがOpen Policy Agentです。


# ポリシーチェック

# a. Open Policy AgentOpen Policy Agent(以降OPA)は、軽量で汎用性のあるポリシーエンジンで、YAMLやJSONのような構造化したデータのポリシーエンジンです。REGOという言語でポリシーを記述します。Kubernetesでは、ConftestやGatekeeperとの組み合わせで利用されることが多いです。
# b. ConftestConftestでは、OPAのREGO言語で記述したポリシーに従ってマニフェストをチェックできます。下図は、マニフェストに「runAsNonRoot」が定義されていればOK、されていなければNGとしてチェックする例です。


# Conftest

# c. GatekeeperGatekeeperは、Kubernetesの認証認可フローの拡張機能であるAdmission Controlから外部Webhookサーバとしてチェック依頼を受け、OPAの定義でチェックを行って登録の許可/拒否を返す仕組みです。


# Gatekeeper

# OPAと組み合わせて、ConftestでCI時にマニフェストのポリシーチェックを行い、GatekeeperでKubernetes登録時にポリシーチェックを行うことでセキュリティを高められます。


# OPA/Conftest/Gatekeeper

# ここで、発表時に含まれていなかった内容を追加します。Software Supply Cahinセキュリティの観点で、インテグリティ保護としてマニフェストに署名を持たせる取り組みがあります。2022年10月末に開催された「KubeCon + CloudNativeCon North America 2022」での発表です。

# Trust But Verify: Bringing Supply Chain Integrity To CD GitOps2022年11月に開催された「Kubernetes Meetup Tokyo #54」においてもRecap Sessionとして発表されました。動画も公開されています。

# 日々進化を続けているので、情報をキャッチアップする必要があります。
# 手元で始める GitOps
# 以下のツール及びサービスを利用したGitOps環境でデモを行いました。

# ・Continuous Integration
# 　- GitHub Actions
# ・Continuous Deliverty
# 　- Argo CD
# ・Kubernetes Cluster
# 　- Oracle Container Engine for Kubernetes (OKE)
# ・Container Registry
# 　- Oracle Cloud Infrastrucutre Registry (OCIR)

# 下図は、デモ構成です。


# デモ構成


# ソースコード更新後、Codeリポジトリ(GitHub)にプッシュ(手動)GitHub Actionsによるビルド、コンテナイメージビルド、OCIRにコンテナイメージプッシュGitHub ActionsによるOCIRへのコンテナイメージプッシュGitHub ActionsによるConfigリポジトリ(GitHub)へのプルリクエストGitHub ActionsによるConftest(ポリシーチェック)の実行プルリクエストをマージ(手動)Argo CDによるConfigリポジトリのポーリングArgo CDによる適用処理Gatekeeperによるポリシーチェックブラウザによる確認(手動)

# デモで使用したソースコードやマニフェストはochacafe-s4-3にあります。


  

# 「Oracle Cloud Hangout Cafe」コミュニティとは
# 「Oracle Cloud Hangout Cafe」(通称「おちゃかふぇ」/以降、OCHaCafe)は、日本オラクルが主催するコミュニティの1つです。定期的に、開発者・エンジニアに向けたクラウドネイティブな時代に身につけておくべきテクノロジーを深堀する勉強会を開催しています。
# OCHaCafeのテーマは、Oracleの製品や技術に特化せず、オープン/デファクト・スタンダードを基準に選定して、参加される方々がいかなるプラットフォームを利用している場合でも、必ずスキルアップに役立てられる内容を趣旨としています。
# 本連載について
# OCHaCafeは、2018年12月から開始して、この4年間、多くの方にご参加いただきました。そして、この4年の間にたくさんの有益なコンテンツが蓄積されました。本連載では、その中からご好評いただいたテーマを選択して、ダイジェスト(全6回)としてお送りします。

# 第1回「CI/CD最新事情」第2回「マイクロサービスの認証・認可とJWT」第3回「Kubernetes Operator 超入門」第4回「Kubernetes のセキュリティ」第5回「実験！カオスエンジニアリング」第6回「Service Mesh がっつり入門！」

# なお、本連載にあたり、各回の内容は実施時の発表内容をベースとしますが、技術の進化により執筆時に内容を変更する場合もあります。その際は、実施時の発表内容とは異なりますので予めご了承ください。
# 連載第1回目の今回は、2021年6月9日に開催された「Oracle Cloud Hangout Cafe Season4 #3『CI/CD 最新事情』」の発表内容に基づいて紹介していきます。
# この発表では、2021年6月時点のCI/CD最新事情として、CI/CDのこれまで、今、これからという時間軸を設けて、CI/CDのおさらいからCIOps、GitOps、Progressive Deliveryまで、デモを交えながら辿りました。
# アジェンダ
# 今回は、以下アジェンダの「CI/CD これまでを見る」「CI/CD 今を見る」「CI/CD これからを見る」を中心に紹介します。

# ・ CI/CD これまでを見る
#  　- CIOps
# 　　- CI/CD おさらい
# 　　- CIOps について
# ・ CI/CD 今を見る
# 　- GitOps
# 　　- GitOps について
# 　　- CI & GitOps
# 　　- 手元で始める GitOps (デモ)
# ・ CI/CD これからを見る
# 　- Progressive Delivery
# 　　- Progressive Delivery について
# ・ Oracle's DevOps
# 　- OCI DevOps Service
# 　- OKE & DevOps
# ・ 最後に
# ・ 参考資料

# 発表資料と動画については、下記のリンクを参照してください。
# 【資料リンク】【動画リンク】
# CI/CD これまでを見る
# 最初に、CI(Continuous Integration/継続的インテグレーション)とCD(Continuous Delivery/継続的デリバリー)について整理します。その上で、CIOpsを理解します。
# CIOps
# CI/CDおさらい
# 1. CIとは?CIは、アプリケーション開発に必要となるビルド、テスト、成果物(アーティファクト)の保存までの工程を自動化して、継続的に行われる状態にすることです。
# コンテナアプリケーション開発における例としては、ソースコードの変更を契機にアプリケーションビルド、アプリケーションテスト、コンテナイメージビルド、コンテナイメージレジストリへのコンテナイメージ格納までの工程を示す場合が多いとされています。
# CI環境を整備することで、早期バグの発見による品質向上や検証時間の短縮によるリリーススピードの向上が見込まれ、開発者が開発に集中できる環境でより良い開発サイクルを実現します。
# 2. CDとは?CDは、CIによって生み出された成果物(アーティファクト)を、実際の環境(ステージングやプロダクション)へ自動的に配置することです。
# コンテナアプリケーション開発における例としては、CIによってコンテナイメージレジストリに格納されたコンテナイメージをデプロイ先となるコンテナプラットフォームに適用する工程です。
# CD環境を整備することで、エンドユーザに影響を与えることなく、安全にアプリケーションのデプロイを実現します。
# 3. CI/CDとは?これまでの流れを整理すると、CI/CDはソースコードの更新を契機にビルド、テストが進行して、アプリケーションが対象となる環境にデプロイされるまでの工程を自動化することだと理解できます。


# CI/CD全体図

# 4. CI/CDのメリットアプリケーション開発において、CI/CDを導入するメリットはいくつかあります。以下に挙げた主な例を基に見ていきましょう。

# リリースサイクルの短縮により、市場ニーズに早く対応できるエンドユーザからのフィードバックを受けて、実装、ビルド、テスト、リリースまでの工程を迅速に実現できます。市場のニーズに早く対応できることで、エンドユーザの満足度、企業競争力や収益の向上も見込めます。
# テスト工程が組まれていることにより、アプリケーションの品質を担保できる早期にバグを発見できることにより品質の向上を計れます。
# 各工程が自動化されたパイプラインのため、全体的な工数削減を見込める各工程が自動化されることによって手動作業が減って工数の削減を見込めます。そして、手動作業が減ることによってヒューマンエラーの回避にもつながります。
# 開発者は開発ロジックに集中できるため、開発効率の向上を見込める開発者が、自動化の恩恵によって、アプリケーション開発ロジックに専念できるようになり、より良い開発サイクルを実現できます。

# CI/CD環境を整備することで、早く正確により良いアプリケーションサービスをエンドユーザに届けることが可能となり、満足度の向上から企業競争力や収益の向上につながります。
# 5. CI/CDの難しさ実際にCI/CDの導入を検討すると、状況にもよりますが簡単にはいかないところもあります。以下に挙げた主な例を基に見ていきましょう。

# CI/CD環境の構築、運用に乗せるまでの試行錯誤実際にCI/CD環境を構築して運用していくことを考えると課題は避けられないと思われます。例えば、開発するアプリケーションの特性、基盤/開発/運用の体制や人数、教育など検討事項は多くあります。
# 多種多様で豊富なCI/CDツールの技術選定CI/CDを実現するツールやサービスはたくさんあるため、基盤/開発/運用者の体制やスキルセットに合わせて検討が必要となります。
# コストの検討開発規模、体制人数、サポートなど状況に合わせて検討が必要となります。
# 定期的なCI/CD環境のメンテナスCI/CD環境は、一度構築して終わりではなく、維持、持続、改善をどうしていくか検討が必要となります。

# もちろん、ここに挙げたことだけではありません。CI/CDの導入において時間やコストも要することは避けられないと思われますが、長い目で見たときに、その分のメリットをどう見込むかが大切なことではないでしょうか。
# CIOpsについて
# CI/CDのおさらいをしたところで、次はCIOpsをベースにコンテナアプリケーション開発におけるCI/CDを見ていきます。
# CIOpsは、これまでのアプリケーション開発からコンテナアプリケーション開発において、広く利用されている手法です。OSS、各ベンダーやクラウドプロバイダーが提供するツールやマネージドサービスがあります。こうしたツールを組み合わせたり、サービスを利用してCI/CD環境を構築します。
# ・プッシュ(Push)型CIOpsは、CI/CDツールやサービスがソースコードの更新を契機に、ビルド、テスト、デプロイの各工程を自動実行します。
# 下図は、コンテナアプリケーション開発において、ソースコード更新してプッシュやマージ処理などをトリガーにCIパイプラインが稼働して、ビルド、テスト、コンテナイメージビルド、コンテナイメージプッシュ、CDパイプラインによってデプロイされる流れです。


# CIOps全体図

# CIOpsは、CI/CDツールやサービスから対象となるプラットフォームにデプロイすることから、プッシュ(Push)型と呼ばれます。デプロイする上でCI/CDツール側に必要となる権限の付与やデプロイ先が増えるごとにその権限を管理するため、セキュリティを十分に考慮する必要があります。
# CI/CD 今を見る
# CIOpsは、CI/CDツールやサービスを利用して、対象となるプラットフォームにデプロイするプッシュ(Push)型でした。ここからは、Kubernetesを基盤としたコンテナプラットフォームが前提となるGitOpsを見ていきます。
# GitOps
# GitOpsについて
# GitOpsという手法が流布し始めたのは、2017年8月に公開されたWeaveworksの「Operations by Pull Request」ブログ記事だと言われています。GitOpsのガイドラインを整理すると、以下となります。

# コードとして記述できるものは、すべてGitに保存及び管理するKubernetesクラスタ上で稼働するコンテナアプリケーションを前提Kubernetes CLIであるkubectlコマンドを直接使用しないKubernetesのオペレータパターンに従うコントローラーを使用

# GitOpsは、デプロイ対象となるプラットフォームがKubernetes基盤上で稼働するコンテナアプリケーションであることが前提で、手動によるコマンド操作をせずにGitを利用してCI/CDを実現する手法です。次項で具体的に見ていきます。
# ・プル(Pull)型GitOpsは、アプリケーションのソースコードを管理するCodeリポジトリとKubernetesで使用するマニフェストを管理するConfigリポジトリを分けて管理します。そして、Kubernetesクラスタ内に配備されたGitOpsオペレータがConfigリポジトリを定期的にポーリングして、マニフェストの更新を検知すると自動的にその更新内容をKubernetesクラスタに適用する仕組みです。
# 下図は、GitOpsの流れです。

# 開発者がソースコードを更新して、Codeリポジトリにプッシュを実行CIパイプラインが稼働して、ビルド、テスト、コンテナイメージビルド、コンテナイメージプッシュの自動実行コンテナイメージに付与された新たなタグをマニフェストに反映後、Configリポジトリにプルリクエスト開発者がプルリクエストをマージGitOpsオペレータが更新を検知後、Kubernetesクラスタに適用



# GitOps全体図

# GitOpsは、Kubernetesを基盤とするコンテナプラットフォームからGitOpsオペレータによってデプロイを実行するため、CIOpsのプッシュ(Push)型とは異なり、プル(Pull)型であることが特徴です。
# GitOpsオペレータは、後述するArgo CDやFluxなどのGitOpsツールもあります。
# ・GitOpsの特徴CodeとConfigリポジトリを分けることで、相互の影響を回避して境界線を明確にできます。また、デプロイに必要となるクレデンシャルなどを外部に持たせることも無いのでセキュリティリスクや管理の煩雑性も抑えられます。


# GitOps特徴(1)

# KubernetesのマニフェストをGitで管理するということは、Configリポジトリにあるマニフェストの内容と実環境の状態が同一になります。Git管理により、マニフェストの履歴管理やロールバック対応など柔軟に実施できるため、実環境の再現性も保証できます。


# GitOps特徴(2)

# マニフェストをGitで管理することによって得られる主な特徴を挙げます。

# デプロイ先に対して「誰が、何時、何」を変更したのか(変更差分)を履歴で追えるデプロイ先をいつでも前の状態に戻すこと(ロールバック)ができるプルリクエストによるレビュー・マージプロセスを通すことで組織ガバナンスを適用できる

# 自動化という点では、CIOpsと同様に手動コマンド操作によるヒューマンエラーの回避、運用コストの低減を見込めます。


 


# CI & GitOps
# 参考までにGitOps環境を構築する上で利用できる主なCIツールを例挙しておきます。CIOpsにおいても利用され続けているCIツールとGitOpsを組み合わせるケースとして主なツールは以下です。

# GitHub ActionsGitLab RunnerCircle CITravis CI

# KubernetesのCustomResourceとしてCIパイプラインをマニフェストで定義できるKubernetesネイティブなツールは以下です。

# TEKTONArgo Workflows

# 主なGitOpsツールは、以下です。

# Argo CDFluxPipeCDJenkis X

# Argoには、Argo Workflows、Argo CD以外にもGitOpsの利便性を高めるツール群もあるので、同じファミリー(Argo Family)として組み合わせて利用できます。
# こうしたツールを組み合わせることでGitOps環境を構築できます。
# ・CI & CD Securityここでは、GitOpsにおいてパイプラインに組み込む、主なセキュリティ対策ナレッジを紹介します。
# 技術の進化に伴い様々なナレッジがありますので、あくまでも一例と捉えてください。
# 1. コンテナイメージビルドコンテナイメージのビルドは、CIパイプラインに組み込む上でセキュアで高速であることが望ましいです。こうした要望に対応するビルドツールもいくつかあります。ここでは、BuildKitとKanikoを紹介します。
# BuildKitは、Dockerfileの命令を並列実行、キャッシュ判定の仕組みで高速にビルドを実行します。そして、非Root / 非Privilegedな実行、DockerfileのRUN 命令を実行する際にseccomp*1を有効化する仕組みによってセキュアにビルドできます。
# Kanikoは、OSやコンテナランタイムに依存せずにコンテナイメージをビルドする場合に有用です。コンテナ内のユーザ空間でビルドすることで、CIで推奨されるDooD*2でのRoot権限で稼働しているホスト上へのセキュリティリスクを回避できます。
# *1:コンテナ内のプロセスが呼び出せるシステムコールを制限できるLinuxカーネルの機能*2:Docker outside of Docker。起動したコンテナからホストのDockerデーモンでイメージビルドする方法
# 2. イメージスキャンコンテナイメージもセキュアであることが望まれます。マネージドサービスのコンテナイメージレジストリに脆弱性検査機能が実装されていますが、CI時に脆弱性スキャンをかけて検査したコンテナイメージをレジストリにプッシュすることでセキュリティを高めることもできます。ここでは、Trivyとdockleを紹介します。
# Trivyは、脆弱性データベースからコンテナイメージを診断します。診断結果をレポートとして出力できます。ワンバイナリでCIに組み込みやすいのも特徴です。
# dockleは、CIS BenchmarkのDockerに関する項目、Dockerfileのベストプラクティスを基に検査できるツールです。そのため、Trivy では検知できない脆弱性を検査できます。
# 3. ポリシーチェックKubernetesのリソースは、マニフェストに定義して登録します。そのマニフェストの内容が正しいか、許可できるものかをチェックしてセキュリティを高める必要があります。
# GitOpsではCI時にConftestというツールを利用してマニフェストをチェックできます。さらに、CD時にGatekeeperというツールを利用してKubernetesクラスタ登録時にチェックできます。その際、ポリシーチェックに利用するポリシーエンジンがOpen Policy Agentです。


# ポリシーチェック

# a. Open Policy AgentOpen Policy Agent(以降OPA)は、軽量で汎用性のあるポリシーエンジンで、YAMLやJSONのような構造化したデータのポリシーエンジンです。REGOという言語でポリシーを記述します。Kubernetesでは、ConftestやGatekeeperとの組み合わせで利用されることが多いです。
# b. ConftestConftestでは、OPAのREGO言語で記述したポリシーに従ってマニフェストをチェックできます。下図は、マニフェストに「runAsNonRoot」が定義されていればOK、されていなければNGとしてチェックする例です。


# Conftest

# c. GatekeeperGatekeeperは、Kubernetesの認証認可フローの拡張機能であるAdmission Controlから外部Webhookサーバとしてチェック依頼を受け、OPAの定義でチェックを行って登録の許可/拒否を返す仕組みです。


# Gatekeeper

# OPAと組み合わせて、ConftestでCI時にマニフェストのポリシーチェックを行い、GatekeeperでKubernetes登録時にポリシーチェックを行うことでセキュリティを高められます。


# OPA/Conftest/Gatekeeper

# ここで、発表時に含まれていなかった内容を追加します。Software Supply Cahinセキュリティの観点で、インテグリティ保護としてマニフェストに署名を持たせる取り組みがあります。2022年10月末に開催された「KubeCon + CloudNativeCon North America 2022」での発表です。

# Trust But Verify: Bringing Supply Chain Integrity To CD GitOps2022年11月に開催された「Kubernetes Meetup Tokyo #54」においてもRecap Sessionとして発表されました。動画も公開されています。

# 日々進化を続けているので、情報をキャッチアップする必要があります。
# 手元で始める GitOps
# 以下のツール及びサービスを利用したGitOps環境でデモを行いました。

# ・Continuous Integration
# 　- GitHub Actions
# ・Continuous Deliverty
# 　- Argo CD
# ・Kubernetes Cluster
# 　- Oracle Container Engine for Kubernetes (OKE)
# ・Container Registry
# 　- Oracle Cloud Infrastrucutre Registry (OCIR)

# 下図は、デモ構成です。


# デモ構成


# ソースコード更新後、Codeリポジトリ(GitHub)にプッシュ(手動)GitHub Actionsによるビルド、コンテナイメージビルド、OCIRにコンテナイメージプッシュGitHub ActionsによるOCIRへのコンテナイメージプッシュGitHub ActionsによるConfigリポジトリ(GitHub)へのプルリクエストGitHub ActionsによるConftest(ポリシーチェック)の実行プルリクエストをマージ(手動)Argo CDによるConfigリポジトリのポーリングArgo CDによる適用処理Gatekeeperによるポリシーチェックブラウザによる確認(手動)

# デモで使用したソースコードやマニフェストはochacafe-s4-3にあります。


 


# 1.GitHub ActionsのCI構成下図は、デモ構成で確認したGitHub Actionsが処理するCIの構成です。


# GitHub Actions CI構成

# GitHub Actionsでは、main.ymlを利用して、CIにおけるワークフローを定義します。その処理内容を確認します。Codeリポジトリ側から見ていきます。
# 【「GitHub Actions main.yaml 1」図の処理内容】

# mainブランチへのプッシュをトリガーとする処理BuildKitによるコンテナイメージビルド処理GitHub Actionsの実行ナンバリングをタグにする処理



# GitHub Actions main.yaml 1

# 【「GitHub Actions main.yaml 2」図の処理内容

# dockleによるイメージ診断処理Trivyによるイメージスキャン処理



# GitHub Actions main.yaml 2

# 【「GitHub Actions main.yaml 3」図の処理内容】

# OCIRへのログイン処理OCIRへのコンテナイメージプッシュ処理



# GitHub Actions main.yaml 3

# 【「GitHub Actions main.yaml 4」図の処理内容】

# GitHubへのログイン処理Configリポジトリからvalues.yamlファイルのクローン処理新規ブランチの作成処理values.yamlファイルのコンテナイメージタグの更新処理Configリポジトリに作成した新規ブランチへのプッシュ処理Configリポジトリへのプルリクエスト処理



# GitHub Actions main.yaml 4

# ここからは、Configリポジトリ側の処理内容です。
# 【GitHub Actions main.yaml 5」図の処理内容】Conftestでは、values.yamlファイル内のコンテナイメージタグにlatest タグがある場合、faildとする定義に従ってポリシーチェックを実行します。

# プルリクエストをトリガーとする処理Conftestによるポリシーチェック処理



# GitHub Actions main.yaml 5

# 以上が、GitHub Actions CIにおけるワークフローの定義です。
# 2.Gatekeeper設定デプロイ対象となるOKEのKubernetesクラスタにGatekeeperの設定をしています。Conftestと同様にlatestタグがある場合、faildとする定義に従ってポリシーチェックを実行します。


# Gatekeeper設定

# 実際のデモについては、こちらの動画リンクから見られます。
# CI/CD これからを見る
# Progressive Deliveryについて
# 1. Progressive Deliveryとは?Progressive Deliveryは、分析しながらデプロイを実行して、その分析結果が失敗と判断された場合は自動的にロールバック、成功と判断された場合はデプロイを継続して行うデプロイ手法です。


# Progressive Delivery

# Continuous Deliveryの次のステップとして、メトリクスを取得して分析を行います。その分析を基に評価して、デプロイするかロールバックするかを判定する仕組みです。
# 2. 主なツールProgressive Deliveryを実現する主なツールは以下です。

# Argo RolloutsFlaggerPipeCD

# 3. Flaggerから見るProgressive DeliveryFlaggerは、KubernetesをプラットフォームにProgressive Delivery を実現するツールです。
# Kubernetes CNIやサービスメッシュプロダクト(Istio、Linkerdなど)と組み合わせたトラフィックシフトによるBlue/Greenデプロイ形式にも対応しています。サービスメッシュプロダクト、Ingress Controller(Nginx、Countorなど)と組み合わせたCanaryリリース、A/Bテストといったデプロイ形式にも対応しています。また、GitOpsツールと組み合わせてProgressive Deliveryを実現できます。
# ここでは、Flaggerと親和性のあるFluxと組み合わせたFlaggerドキュメント情報を基に見ていきます。
# FluxからKubernetesクラスタにデプロイが実行される際に、Prometheusからメトリクスを取得して分析しながら、異なるバージョンのアプリケーションへのトラフィック量を変更(Canaryリリース)していきます。


# Flagger Progressive Delivery【出典】「Flagger Official Documents - Introduction」

# Flaggerには、Webhookを利用した分析の拡張機能があります。以下8種類のHookがあり、そのレスポンスコードによって切り替えます(【参考】「Flagger Official Documents - Webhooks」)。

# confirm-rolloutpre-rolloutrolloutconfirm-traffic-increaseconfirm-promotionpost-rolloutrollbackevent

# このことを踏まえて、IstioとFlaggerを組み合わせたチュートリアルが公式サイトにあります。
# カナリア分析が開始されると、FlaggerはトラフィックをCanaryにルーティングする前にpre-rollout webhookを呼び出します。カナリア分析は、HTTPメトリクスとロールアウト フックを毎分検証しながら5分間実行されます。


# Istio Canary Deployments 1【出典】「Flagger Official Documents - Istio Canary Deployments」

# 下図は、Flaggerが分析しながらトラフィック量を変更して、最終的に最新バージョンにシフトして行く流れです。


# Istio Canary Deployments 2【出典】「Flagger Official Documents - Istio Canary Deployments」

# おわりに
# CIOps、GitOps、Progressive Deliveryの流れで、コンテナアプリケーション開発をベースにCI/CDのこれまで、今、これからを見てきました。あくまでも、これらは技術手段の一部です。こうした技術手段を活かした実現する目的を忘れてはいけません。CI/CD環境を整備することで、スピード(Agility)、信頼性(Reliability)を高めて、品質及び生産性の高いアプリケーション開発の実現を目指します。
# これまで時間やコストを要していた箇所を改善し、より良いビジネスロジックに時間やコストを傾けて、リリースサイクルを速め、エンドユーザに最高品質のサービスの提供につなげていきます。そして、最終的にエンドユーザの満足度、企業価値や収益の向上を見込むことができるのではないでしょうか。


# 最後に
# """)

In [ ]:
@observe()
def insert_data(data: dict | list[dict]) -> dict:
    res = client.insert(
        collection_name=collection_name,
        data=data
    )
    return res

In [ ]:
files = glob.glob("../data/text/*.txt")
for file in files:
    with open(file) as f:
        content = f.read()
        data = create_data_by_character_text_splitter(content)
        _ = insert_data(data)

In [ ]:
index_params = MilvusClient.prepare_index_params()

index_params.add_index(
    field_name="vector",
    metric_type="COSINE",
    index_type="IVF_FLAT",
    index_name="vector_index",
    params={ "nlist": 128 }
)

client.create_index(
    collection_name=collection_name,
    index_params=index_params
)

In [ ]:
client.load_collection(collection_name=collection_name)
res = client.get_load_state(collection_name=collection_name)
print("load state: ", res)

In [ ]:
query_vectors = embedding(["OCHaCafeとは何ですか？"])

res = client.search(
    collection_name="ochacafe_collection",
    data=query_vectors,
    limit=3,
    output_fields=["text", "subject"]
)

print(res)